In [1]:
# !pip install torchinfo
# !pip install torcheval

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')


In [3]:
# %cd /content/drive/MyDrive/2023_AI_new

In [4]:
import datetime
import os
import time
from tqdm import tqdm
import torch
import h5py
import numpy as np
from torchinfo import summary
import math
from torch.autograd import Variable
import pickle

In [5]:
# custom module
import model, pamap_dataloader, train_model, test_model
import importlib
importlib.reload(model)
importlib.reload(pamap_dataloader)
importlib.reload(train_model)
importlib.reload(test_model)
from model import *
from data import *
from pamap_dataloader import *
from train_model import train
from test_model import test

# Parameters

In [6]:
config = {
    "root" : "./data/pickles/",
    "train_feature" : "PAMAP2_feature.pickle",
    "train_target" : "PAMAP2_target.pickle",
    "test_feature" : "PAMAP2_feature_test.pickle",
    "test_target" : "PAMAP2_target_test.pickle",
    "labelencodedict" :  { 1 : 0,
                   3 : 1,
                   4 : 2,
                   6 : 3,
                   7 : 4,
                   16 : 5,
                   17 : 6
                   },
    "epochs" : 3000,
    "learning_rate" : 1e-4,
    # model 
    "first_out_channels" : 32,
    "first_kernel_size" : 5,
    "second_in_channels" : 32,
    "second_out_channels" : 16,
    "second_kernel_size" : 3,
    "dim_model" : 128,
    "dim_inner" : 256,
    "num_heads" : 8,
    "dropout_rate" : 0.15,
    "squeeze_factor" : 2,
    "sec_kernel_size" : 3,
    "sec_stride" : 1,
    "N" : 3,
    "gamma" : 2,
    "u" : 2,     
    "weight_decay" : 0.1,
    "device" : torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

# Main

In [7]:
if __name__ == "__main__":
    
    epochs = config["epochs"]
    learning_rate = config["learning_rate"]
    weight_decay = config["weight_decay"]
    X_train, y_train, train_y, X_test, y_test, test_y = load_dataset(config)
    
    X_train = X_train.double()
    X_test = X_test.double()
    config["num_samples"] = X_train.shape[0]
    config["first_in_channels"] = X_train.shape[1]
    config["L"] = X_train.shape[2]
    config["length"] = X_train.shape[2]
    
    md = Net(
                dim_model = config["dim_model"],
                gamma = config["gamma"],
                u = config["u"],
                ys = y_train,
                first_in_channels = config["first_in_channels"],
                first_out_channels = config["first_out_channels"],
                first_kernel_size = config["first_kernel_size"],
                second_in_channels = config["second_in_channels"],
                second_out_channels = config["second_out_channels"],
                second_kernel_size = config["second_kernel_size"],
                N = config["N"],  #slice
                L = config["L"], # slice
                dim_inner = config["dim_inner"],
                num_samples = config["num_samples"], 
                length = config["length"], 
                num_heads = config["num_heads"],
                dropout_rate = config["dropout_rate"],
                squeeze_factor = config["squeeze_factor"],
                sec_kernel_size = config["sec_kernel_size"],
                sec_stride = config["sec_stride"],
                num_classes = len(config["labelencodedict"].keys())).to(config['device'])
    optimizer = torch.optim.Adam(md.parameters(),weight_decay=weight_decay, lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()
    
    train_loss, train_acc, train_f1  = train(X_train, train_y, optimizer, model = md, epochs = epochs, f1=True)
    md.ys = y_test
    test_loss, test_acc, test_f1  = test(X_test, test_y, model = md, f1 = True)

C:\Users\yuins\Hanyang_23_1st\AI\model.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)
C:\Users\yuins\Hanyang_23_1st\AI\model.py:242: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)


Epoch: 0, Loss: 0.8832, Train acc: 97.5510%, F1 macro: 0.9166
Epoch: 1, Loss: 0.7906, Train acc: 98.3673%, F1 macro: 0.9444
Epoch: 2, Loss: 0.5250, Train acc: 95.1020%, F1 macro: 0.8297
Epoch: 3, Loss: 0.5773, Train acc: 98.3673%, F1 macro: 0.9423
Epoch: 4, Loss: 0.4441, Train acc: 99.1837%, F1 macro: 0.9711
Epoch: 5, Loss: 0.3617, Train acc: 98.3673%, F1 macro: 0.9423
Epoch: 6, Loss: 0.3123, Train acc: 99.1837%, F1 macro: 0.9711


KeyboardInterrupt: 

In [8]:
md.ys = y_test
test_loss, test_acc, test_f1  = test(X_test, test_y, model = md, f1 = True)

TEST, Loss: 1.2436, Test acc: 91.2088%, F1 macro: 0.6095


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10,6))
plt.ylim([-0.1, 1.5])
plt.title("Train Loss", fontsize = 20)
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.axhline(1, alpha = 0.1, color = "black", linestyle = "--")
plt.plot([i.cpu().detach().numpy() for i in train_loss])


plt.figure(figsize = (10,6))
plt.ylim([90, 102])
plt.title("Train Accuracy", fontsize = 20)
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Accuracy", fontsize=15)
plt.axhline(100, alpha = 0.1, color = "black", linestyle = "--")
plt.plot([i.cpu().detach().numpy() for i in train_acc])


In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import seaborn as sns

model_dimreduce = TSNE(n_components=2, learning_rate='auto',
                  init='pca', perplexity=3)
scaler = StandardScaler()

for i in range(len(train_representation)):
    _rep = train_representation[i].cpu().detach().numpy()
    _class = train_prototypes[i].squeeze(dim =1).cpu().detach().numpy()

    _rep_tsne = model_dimreduce.fit_transform(_rep)
    _class_tsne = model_dimreduce.fit_transform(_class)

    _rep_tsne = scaler.fit_transform(_rep)
    _class_tsne = scaler.transform(_class)
    plt.title("Epoch {0}".format((i * 250)), fontsize=20)
    sns.scatterplot(x =_rep_tsne[:, 0], y = _rep_tsne[:, 1], label = "Representation")
    sns.scatterplot(x =_class_tsne[:, 0], y = _class_tsne[:, 1], marker = "*", label = "Class prototypes", s=100)
    plt.legend()
    plt.show()